In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-vine").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [ ]:
vine_df.count()

10319090

In [ ]:
# column data type code from https://sparkbyexamples.com/pyspark/pyspark-find-datatype-column-names-of-dataframe/
for col in vine_df.dtypes:
  print(col[0] + ", " + col[1])

review_id, string
star_rating, string
helpful_votes, int
total_votes, int
vine, string
verified_purchase, string


In [ ]:
# filter code from https://sparkbyexamples.com/pyspark/pyspark-where-filter/
vine_df = vine_df.filter(vine_df.total_votes >= 20)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R28WZIA84DUP6C|          2|            4|         43|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|     

In [ ]:
vine_df.count()

161595

In [ ]:
vine_df = vine_df.filter((vine_df.helpful_votes)/(vine_df.total_votes) >= 0.5)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
| RTX63ENBKWCXF|          1|           28|         35|   N|     

In [ ]:
vine_df.count()

114309

In [ ]:
stars_count = vine_df.filter(vine_df.star_rating == 5).count()
stars_count

51998

In [ ]:
paid_vine_df = vine_df.filter(vine_df.vine == 'Y')
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R397PIJYYVJ7PD|          4|           31|         32|   Y|                N|
| RP6Y2CTIFUYD0|          2|           31|         32|   Y|                N|
|R1FYRNM5U0S2VZ|          3|           20|         24|   Y|                N|
| R1ME19UY4UNAO|          5|           53|         57|   Y|                N|
|R1JW7ZHT2VLT1E|          3|           40|         45|   Y|                N|
| R98UYXY43TURV|          4|           46|         50|   Y|                N|
|R34VXM3EJDS78Z|          3|          374|        410|   Y|                N|
|R1QX8UOW5JO6L0|          4|           20|         21|   Y|                N|
| R5Y5SXCRQ2GNH|          3|           34|         37|   Y|     

In [ ]:
paid_vine_df.count()

5012

In [ ]:
paid_5_stars_df = paid_vine_df.filter(paid_vine_df.star_rating == 5)
paid_5_stars_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
| R1ME19UY4UNAO|          5|           53|         57|   Y|                N|
| RCLSDAIJR6FUG|          5|          113|        123|   Y|                N|
|R30QKO5AIO2828|          5|           17|         20|   Y|                N|
|R3TPSU0ZG7K5RA|          5|           34|         37|   Y|                N|
|R2TSF7WN3DY921|          5|           22|         26|   Y|                N|
|R1T5WLUGSGBKT2|          5|           34|         36|   Y|                N|
|R3A5Z6PHLU5TNP|          5|           43|         45|   Y|                N|
| R5OWLL4LN7FOB|          5|           21|         23|   Y|                N|
|R1E37AORP2EI9G|          5|           61|         65|   Y|     

In [ ]:
paid_5_stars_df.count()

2031

In [ ]:
paid_stars_count = paid_5_stars_df.count()
paid_vine_count = paid_vine_df.count()
paid_percentage = (paid_stars_count / paid_vine_count) * 100
paid_percentage

40.52274541101357

In [ ]:
unpaid_vine_df = vine_df.filter(vine_df.vine == 'N')
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
| RTX63ENBKWCXF|          1|           28|         35|   N|                N|
|R38I0D0J8O6LB6|          5|          101|        102|   N|     

In [ ]:
unpaid_vine_df.count()

109297

In [ ]:
unpaid_5_stars_df = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5)
unpaid_5_stars_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
|R38I0D0J8O6LB6|          5|          101|        102|   N|                Y|
|R16H89H2ZPISOL|          5|           66|         73|   N|                Y|
|R1O92J5SV8J8G6|          5|          100|        102|   N|                Y|
| RERKINI37W6KC|          5|           28|         31|   N|                Y|
|R37RUX3TXFJN24|          5|           52|         58|   N|     

In [ ]:
unpaid_5_stars_df.count()

49967

In [ ]:
unpaid_stars_count = unpaid_5_stars_df.count()
unpaid_vine_count = unpaid_vine_df.count()
unpaid_percentage = (unpaid_stars_count / unpaid_vine_count) * 100
unpaid_percentage

45.716716835777746